- 发生异常: AssertionError
Path ./original_model/smplx/SMPLX_MALE.npz does not exist! SMPLX_MALE.npz
    - betas 10
    - pose 87
    - gender
    - model
    - vertices 10475,3

In [ ]:
import smplx

def get_bodymodels(
        model_path, 
        model_type, 
        device, 
        batch_size=1, 
        num_pca_comps=12
    ):

    models = {}

    # model parameters for self-contact optimization
    model_params = dict(
        batch_size=batch_size,
        model_type=model_type,
        create_body_pose=True,
        create_transl=False,
        create_betas=False,
        create_global_orient=False,
        create_left_hand_pose=True,
        create_right_hand_pose=True,
        use_pca=True,
        num_pca_comps=num_pca_comps,
        return_full_pose=True,
    )
    
    # create smplx model per gender
    for gender in ['male', 'female', 'neutral']:
        models[gender] = smplx.create(
            model_path=model_path,
            gender=gender,
            **model_params
        ).to(device)
    
    return models

In [2]:
import numpy as np

class Struct(object):
    def __init__(self, **kwargs):
        for key, val in kwargs.items():
            setattr(self, key, val)

smplx_path = './original_model/smplx/smplx_pose1.npz'
model_data = np.load(smplx_path, allow_pickle=True)
data_struct = Struct(**model_data)

shapedirs = data_struct.shapedirs

print(model_data)

AttributeError: 'Struct' object has no attribute 'shapedirs'

In [4]:
import torch
print(torch.version.cuda)

11.6


In [6]:
use_cuda = True
dtype = torch.LongTensor
diag_ind_x = torch.arange(0, 10).type(dtype)

In [ ]:
1. 与OBB盒计算winding numbers
    1. 根据不同身体部位的顶点构建OBB盒，顶点有对应的部位segment索引
    2. 同一个segment的顶点，与其他不同segment的OBB盒计算winding numbers
    3. 如果在某个OBB盒内部，就与该segment的顶点计算一次精确的winding numbers，对每个OBB都计算，直到结束or在内部

SelfContact 479 -> 1664
load_model_param 1642 -> 2536
loss.configure 2529 -> 4876
segment_points_scopti 4794 -> 6047

v2v占用200MB
但是如果v2v